In [ ]:

def calc_ln(params):
    return params["D"]

def calc_qkv_proj(params):
    return params["D"]

def calc_qk_mult(params):
    return 2 * params["D"]

def calc_softmax(params):
    return params["num_heads"] * params["context_length"]**2
    
def calc_w_v(params):
    return params["num_heads"] * params["context_length"]**2 + params["D"]
    
def calc_out_proj(params):
    return params["D"]
    
def calc_attn(params):
    return calc_qkv_proj(params) + calc_qk_mult(params) + calc_softmax(params) + calc_w_v(params) + calc_out_proj(params)

def calc_ffn(params):
    return 9 * params["D"]
    
def calc_block_size(params):
    res = 2*calc_ln(params) + calc_attn(params) + calc_ffn(params)
    assert res == 16 * params["D"] + 2 * params["num_heads"] * params["context_length"]**2
    return res
    
def calc_lm_head(params): 
    return params["D"]
    
def calc_cross_entropy(params):
    return params["context_length"] * params["vocab_size"]
    
def calc_act(params):
    d = params["context_length"] * params["d_model"]
    params["D"] = d
    act = params["num_layers"] * calc_block_size(params) + calc_ln(params) + calc_lm_head(params) + calc_cross_entropy(params)
    assert act == params["num_layers"] * (16 * params["D"] + 2 * params["num_heads"] * params["context_length"] ** 2) + 2 * params["D"] +\
        params["context_length"] * params["vocab_size"]
    return act

def calc_params(params):
    emb = params["vocab_size"] * params["d_model"]
    block = 4 * params["d_model"]**2 + 12 * params["d_model"]**2
    lm_head = params["d_model"] * params["vocab_size"]
    
    b = emb + params["num_layers"] * block + lm_head
    
    assert b == 2 * params["vocab_size"]*params["d_model"] + 16*params["num_layers"]*params["d_model"]**2
    return b

   
def calc_memory(params):
    act = calc_act(params)
    pars = calc_params(params)

    a = 4 * act
    assert a == 4 * (params["num_layers"] * (16 * params["context_length"]*params["d_model"] + 
                                             2 * params["num_heads"] * params["context_length"]**2) + 2 * params["context_length"]
                     * params["d_model"] + params["context_length"] * params["vocab_size"])
    b = 16 * pars
    assert b == 16 * (2 * params["vocab_size"]*params["d_model"] + 16 * params["num_layers"]*params["d_model"]**2)
                      
    print(f"Memory in bytes = {a:,}*batch_size+{b:,}")

    mem = 80 * 1024**3
    print(f"Number of batches to fit into 80G: {(mem - b) / a:.1f}")  # Well, obviously 3.

In [ ]:
PARAMS = dict(
vocab_size=50_257,
context_length=1_024,
num_layers=48,
d_model=1_600,
num_heads=25)

calc_memory(PARAMS)